In [10]:
import numpy as np 
X, Y = [], []

def grid_to_tuple(grid):
    tup = []
    for x in grid:
        if x == '-':
            tup.append(0)
        elif x == 'x':
            tup.append(1)
        elif x == 'o':
            tup.append(-1)
        else:
            assert False
    return tuple(tup)

for line in open('train_data.txt').readlines():
    grid, result = line.strip().split()
    X.append(grid_to_tuple(grid))
    Y.append(result == 'True')

X = np.array(X)[:25]
Y = np.array(Y)[:25]

In [11]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_small = pca.fit_transform(X)

In [12]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

features = MinMaxScaler(feature_range=(0, np.pi)).fit_transform(X)

train_features, test_features, train_labels, test_labels = train_test_split(
    X, Y, train_size=15, shuffle=False
)

In [13]:
# number of qubits is equal to the number of features
num_qubits = 2

# number of steps performed during the training procedure
tau = 100

# regularization parameter
C = 1000

In [18]:
from qiskit import *
IBMQ.providers()
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.utils import algorithm_globals
from qiskit.circuit.library import ZFeatureMap
from qiskit_machine_learning.algorithms import PegasosQSVC


from qiskit.visualization import plot_histogram

IBMQ.save_account('53fbe5f38c448c1c5d19a96fa47bd10ee765b1a9721453022212a2f5b3b29c0fb75a665962ef38dd5fde63cd26ddb2cd9d86fe78c9a37b947ff91e6bf911b0ea', overwrite=True)

IBMQ.load_account()

provider = IBMQ.get_provider(hub = 'ibm-q')

device = provider.backends()
print(device)

device = provider.get_backend('ibmq_quito')


ibmqfactory.load_account:WARNING:2023-01-29 02:42:02,292: Credentials are already in use. The existing account in the session will be replaced.


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_manila') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_nairobi') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_oslo') from IBMQ(hub='ibm-q', group='open', project='main')>]


In [19]:
from qiskit_machine_learning.kernels import QuantumKernel


algorithm_globals.random_seed = 12345

feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)

pegasos_backend = QuantumInstance(backend = device)

qkernel = QuantumKernel(feature_map=feature_map, quantum_instance=pegasos_backend)

pegasos_qsvc = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)

pegasos_qsvc.fit(train_features, train_labels)


# testing
pegasos_score = pegasos_qsvc.score(test_features, test_labels)
print(f"PegasosQSVC classification test score: {pegasos_score}")



TranspilerError: 'Number of qubits (9) in circuit-436 is greater than maximum (5) in the coupling_map'